## Set the path of where the csv files are located, as well as initial import statements

In [1]:
csv_data_path = "" # **PLACE CSV FILE HERE**
sep = "\t" # csv delimiter used
small_model = True # use smaller ML model for better performance, less accuracy

In [2]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Functions to handle data loading and data writing

In [3]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_file(directory: Path):
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield (load_data(data_file.resolve()), data_file.name)

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [4]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
data = load_data(csv_data_path)['content']
data.head()

0    Hi,  I'm looking for monthly average mobile do...
1    The latest ookla speedtest app (just for apple...
2    We had some unusual results come back today fr...
3    Hi there,  I am working with a group that is i...
4    Hi, It looks like the -server option in ndt7-c...
Name: content, dtype: object

# Content Preprocessing

In [5]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data[0][:10]

NameError: name 'data' is not defined

In [7]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
tokenized_data[0][:10]

['look',
 'monthly',
 'average',
 'mobile',
 'download',
 'speed',
 'datum',
 'country',
 'specifcally',
 'onwards']

# Generate id2word dictionary

In [8]:
id2word = corpora.Dictionary(tokenized_data)

corpus = []

for text in tokenized_data:
    new = id2word.doc2bow(text)
    corpus.append(new)

corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

# LDA Visualization

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=30,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarn

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.271088  0.321027       1        1  23.452643
3      0.458450 -0.016092       2        1  13.313981
5     -0.180690  0.395276       3        1  12.681506
7      0.003566  0.414915       4        1  11.168300
23     0.360997 -0.293879       5        1  11.081298
16     0.357495  0.132768       6        1   7.743611
9      0.114880  0.309229       7        1   3.749058
8     -0.170723 -0.348844       8        1   2.412309
11    -0.321994  0.120213       9        1   2.003691
4      0.160560  0.112501      10        1   1.335209
29     0.165503 -0.078450      11        1   1.199092
24    -0.114052  0.202832      12        1   1.187955
6     -0.199189 -0.154371      13        1   1.156888
14    -0.162109  0.098278      14        1   1.140861
25     0.026218 -0.240056      15        1   1.083942
26    -0.231249  0.003764      16        1   0.922946
2     -0.252632 -0.074203      17        1   0.809639
19    -0.111000 -0.172228      18        1   0.767583
18     0.043460  0.053608      19        1   0.703960
10     0.070419 -0.125999      20        1   0.699128
17    -0.045578 -0.140189      21        1   0.505944
1      0.014597 -0.140108      22        1   0.414488
20    -0.006810 -0.053958      23        1   0.224391
13    -0.040088 -0.051466      24        1   0.081556
22    -0.039412 -0.043001      25        1   0.058082
21    -0.034334 -0.046297      26        1   0.020960
28    -0.034343 -0.046317      27        1   0.020632
27    -0.034343 -0.046317      28        1   0.020226
15    -0.034343 -0.046317      29        1   0.020202
0     -0.034344 -0.046318      30        1   0.019920, topic_info=            Term        Freq       Total Category  logprob  loglift
49          test  239.000000  239.000000  Default  30.0000  30.0000
137          web  123.000000  123.000000  Default  29.0000  29.0000
245         type   83.000000   83.000000  Default  28.0000  28.0000
1466        snap   94.000000   94.000000  Default  27.0000  27.0000
749        state  104.000000  104.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
874   acceptance    0.001194    5.470331  Topic30  -7.8721   0.0912
897     delivery    0.001194   10.248933  Topic30  -7.8721  -0.5367
963       unique    0.001194    7.567502  Topic30  -7.8721  -0.2334
70      standard    0.001194   11.995688  Topic30  -7.8721  -0.6941
519         page    0.001194   14.744822  Topic30  -7.8721  -0.9004

[1518 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         2  0.511070       able
0         9  0.046461       able
0        11  0.418148       able
757       5  0.951294   abstract
1119      3  0.810917  abundance
...     ...       ...        ...
341      12  0.148248       year
138       4  0.331863        yet
138      11  0.221242        yet
138      18  0.442484        yet
428      15  0.817199       zoom

[1305 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 4, 6, 8, 24, 17, 10, 9, 12, 5, 30, 25, 7, 15, 26, 27, 3, 20, 19, 11, 18, 2, 21, 14, 23, 22, 29, 28, 16, 1])